# Replication Draft

## Pull from Markit and construct CDS-related columns

In [1]:
import wrds
WRDS_USERNAME="ygxu"

db = wrds.Connection(wrds_username = WRDS_USERNAME)

## this is the df that I finally used 
query="""
    SELECT ticker, redcode, DATE_TRUNC('month', date) AS month, MIN(date) AS first_trade_date, AVG(parspread) AS midspread
    FROM markit.cds2001
    WHERE tenor = '5Y'
    GROUP BY month, ticker, redcode
    """
#db.raw_sql(query)
cds01_df = db.raw_sql(query)

Loading library list...
Done


In [2]:
cds01_df.head(5)

,ticker,redcode,month,first_trade_date,midspread
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089
1,CROATI,248DEC,2001-08-01 00:00:00-04:00,2001-08-01,0.016329
2,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838
3,LNC,5I759F,2001-11-01 00:00:00-05:00,2001-11-01,0.004350
4,CITOH,4ABC88,2001-07-01 00:00:00-04:00,2001-07-02,0.019080


In [3]:
cds01_df["prev_spread"] = cds01_df.groupby("ticker")["midspread"].shift(1)

## Calculate RD

In [ ]:
import pandas as pd 
file_path = "../data_manual/feds200628.csv"
df = pd.read_csv(file_path,skiprows=9)
df["Date"] = pd.to_datetime(df["Date"])

In [11]:
df.head(5)

,Date,BETA0,BETA1,BETA2,BETA3,SVEN1F01,SVEN1F04,SVEN1F09,SVENF01,SVENF02,...,SVENY23,SVENY24,SVENY25,SVENY26,SVENY27,SVENY28,SVENY29,SVENY30,TAU1,TAU2
0,1961-06-14,3.917606,-1.277955,-1.949397,0.0,3.8067,3.9562,NaN,3.5492,3.8825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.339218,-999.99
1,1961-06-15,3.978498,-1.257404,-2.247617,0.0,3.8694,4.0183,NaN,3.5997,3.9460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.325775,-999.99
2,1961-06-16,3.984350,-1.429538,-1.885024,0.0,3.8634,4.0242,NaN,3.5957,3.9448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.348817,-999.99
3,1961-06-19,4.004379,-0.723311,-3.310743,0.0,3.9196,4.0447,NaN,3.6447,3.9842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.282087,-999.99
4,1961-06-20,3.985789,-0.900432,-2.844809,0.0,3.8732,4.0257,NaN,3.5845,3.9552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.310316,-999.99


In [5]:
rf_columns = ["SVENY01", "SVENY02", "SVENY03", "SVENY04","SVENY05","SVENY06", "SVENY07","SVENY08","SVENY09", "SVENY10"]
maturities = [i for i in range(0, 11)]

In [19]:
file_path = "../data_manual/DGS3MO.csv"
df3 = pd.read_csv(file_path)

file_path = "../data_manual/DGS6MO.csv"
df6 = pd.read_csv(file_path)

df_treasury = pd.merge(df3, df6, on = "observation_date")
df_treasury.head(5)
df_treasury["Date"] = pd.to_datetime(df_treasury["observation_date"])

In [20]:
df_treasury = df_treasury.drop(columns="observation_date")
df_treasury.head(5)

,DGS3MO,DGS6MO,Date
0,17.01,17.17,1981-09-01
1,16.65,17.32,1981-09-02
2,16.96,17.42,1981-09-03
3,16.64,17.37,1981-09-04
4,NaN,NaN,1981-09-07


In [85]:
rf_columns = ["Date", "SVENY01", "SVENY02", "SVENY03", "SVENY04","SVENY05","SVENY06", "SVENY07","SVENY08","SVENY09", "SVENY10"]
df_rf = df[rf_columns].merge(df_treasury, on = "Date")
df_rf.head(5)

,Date,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10,DGS3MO,DGS6MO
0,1981-09-01,16.1930,15.7597,15.3419,15.0393,14.8322,14.6891,14.5866,14.5104,14.4516,14.4047,17.01,17.17
1,1981-09-02,16.2582,15.8046,15.3635,15.0539,14.8447,14.6996,14.5946,14.5156,14.4541,14.4049,16.65,17.32
2,1981-09-03,16.3433,15.8456,15.3940,15.0987,14.9074,14.7772,14.6838,14.6137,14.5592,14.5156,16.96,17.42
3,1981-09-04,16.4590,15.8969,15.4284,15.1286,14.9355,14.8042,14.7100,14.6392,14.5842,14.5401,16.64,17.37
4,1981-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df_rf = df_rf.sort_values(by='Date')

In [87]:
quarter_start_dates = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='QS')

In [88]:
days_since_start = (df['Date'] - df['Date'].min()).dt.days
quarter_days_since_start = (quarter_start_dates - df['Date'].min()).days

In [89]:
df_rf.columns[1:].to_list()

['SVENY01',
 'SVENY02',
 'SVENY03',
 'SVENY04',
 'SVENY05',
 'SVENY06',
 'SVENY07',
 'SVENY08',
 'SVENY09',
 'SVENY10',
 'DGS3MO',
 'DGS6MO']

In [90]:
df_rf.head()

,Date,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10,DGS3MO,DGS6MO
0,1981-09-01,16.1930,15.7597,15.3419,15.0393,14.8322,14.6891,14.5866,14.5104,14.4516,14.4047,17.01,17.17
1,1981-09-02,16.2582,15.8046,15.3635,15.0539,14.8447,14.6996,14.5946,14.5156,14.4541,14.4049,16.65,17.32
2,1981-09-03,16.3433,15.8456,15.3940,15.0987,14.9074,14.7772,14.6838,14.6137,14.5592,14.5156,16.96,17.42
3,1981-09-04,16.4590,15.8969,15.4284,15.1286,14.9355,14.8042,14.7100,14.6392,14.5842,14.5401,16.64,17.37
4,1981-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
import numpy as np
rate_columns = df_rf.columns[1:].to_list()
df_rf[rate_columns] = df_rf[rate_columns].replace([np.inf, -np.inf], np.nan) 
df_rf[rate_columns] = df_rf[rate_columns].fillna(method="ffill").fillna(method="bfill")


C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_26460\860331299.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_rf[rate_columns] = df_rf[rate_columns].fillna(method="ffill").fillna(method="bfill")


In [94]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

def convert_maturity_labels(columns):
    """
    Converts maturity labels into numerical values in months.
    Example: 'SVENY05' → 60 months, 'DGS3MO' → 3 months.
    """
    maturity_map = {}
    for col in columns:
        if col.startswith("SVENY"):  # Convert 'SVENYXX' → months
            years = int(col[-2:])  # Extract last 2 digits
            maturity_map[col] = years * 12
        elif col == "DGS3MO":
            maturity_map[col] = 3
        elif col == "DGS6MO":
            maturity_map[col] = 6
        else:
            raise ValueError(f"Unknown maturity label: {col}")
    
    # Sort maturities by increasing order
    sorted_maturities = sorted(maturity_map.items(), key=lambda x: x[1])
    
    # Return sorted column names and maturity values
    sorted_columns, sorted_maturity_values = zip(*sorted_maturities)
    
    return list(sorted_columns), np.array(sorted_maturity_values)


def extrapolate_rates_decaying(rates, decay_type='exp', alpha=0.05):
    """
    Interpolates and extrapolates risk-free rates to quarterly maturities 
    while applying a decaying weight based on maturity.

    Parameters:
    - rates: DataFrame containing rate maturities as columns.
    - decay_type: 'exp' for exponential decay, 'inv' for inverse maturity weighting.
    - alpha: Decay rate for exponential weighting.

    Returns:
    - DataFrame with quarterly-maturity rates for each time index.
    """
    extrapolated_rates_df = pd.DataFrame(
        columns=np.arange(3, 121, 3), index=rates.index
    )

    # Convert column labels to numerical maturities in months (sorted)
    sorted_columns, maturities = convert_maturity_labels(rates.columns)

    # Ensure rates are reordered based on sorted maturities
    rates = rates[sorted_columns]

    # Define weight function based on decay type
    if decay_type == 'exp':
        weights = np.exp(-alpha * maturities)  # Exponential decay
    elif decay_type == 'inv':
        weights = 1 / maturities  # Inverse maturity weighting
    else:
        raise ValueError("Invalid decay_type. Choose 'exp' or 'inv'.")

    # Normalize weights so they sum to 1
    weights /= weights.sum()

    for index, row in rates.iterrows():
        rates_values = row.values * weights  # Apply decaying weights

        # Cubic spline interpolation (maturities are now strictly increasing)
        cs = CubicSpline(maturities, rates_values, bc_type='natural')

        # Generate quarterly maturities (3, 6, 9, ..., 120 months)
        quarterly_maturities = np.arange(3, 121, 3)
        extrapolated_rates = cs(quarterly_maturities)

        extrapolated_rates_df.loc[index] = extrapolated_rates

    return extrapolated_rates_df

# Example Usage
decay_type = 'exp'  # Choose 'exp' for exponential or 'inv' for inverse maturity
alpha = 0.05  # Controls decay speed in exponential weighting

extrapolated_rates = extrapolate_rates_decaying(df_rf.iloc[:,1:], decay_type=decay_type, alpha=alpha)


# Example Usage
decay_type = 'exp'  # Choose 'exp' for exponential or 'inv' for inverse maturity
alpha = 0.05  # Controls decay speed in exponential weighting

extrapolated_rates = extrapolate_rates_decaying(df_rf.iloc[:,1:], decay_type=decay_type, alpha=alpha)


In [99]:
import numpy as np
import pandas as pd

# Step 1: Extract interpolated quarterly risk-free rates
df_interpolated = extrapolated_rates  # Already computed interpolated rates

# Step 2: Compute λ (hazard rate) from CDS spreads
CDS = 0.01  # Example CDS spread (1%)
L = 0.6  # Loss given default (60%)
lambda_value = 4 * np.log(1 + CDS / (4 * L))  # Compute λ from CDS spreads

# Step 3: Compute RD_t for each quarter separately
rd_per_quarter = []

for t in range(1, len(df_interpolated) + 1):  # Iterate over each quarter
    quarterly_rates = df_interpolated.iloc[:t].mean(axis=1).values  # Use rates up to quarter t
    
    RD_t = (1 / 4) * sum(
        np.exp(-j * lambda_value / 4) * np.exp(-j * quarterly_rates[j-1] / 4)
        for j in range(1, t + 1)
    )
    
    rd_per_quarter.append(RD_t)

# Convert to DataFrame
df_RD = pd.DataFrame({"Date": df_interpolated.index, "RD_t": rd_per_quarter})
print(df_RD.head())  # View first few results


   Date      RD_t
0     0  0.199798
1     1  0.359493
2     2  0.486591
3     3  0.588033
4     4  0.668996
